### FIAT: Framework for Iterative and Agnostic Testing of Hydrological Models

This notebook demonstrates how to run a simple test case using **FIAT** – the *Framework for Iterative and Agnostic Testing of Hydrological Models*. FIAT is designed to help hydrologists and model developers systematically evaluate models under a variety of conditions, while remaining agnostic to the specific model implementation or numerical core. The goal is to provide a common, reproducible workflow for running and analyzing iterative test cases (e.g., autocalibration experiments) across different hydrological modeling platforms.

In this example, we will walk through the typical FIAT workflow for a single test case:

1. Defining or selecting a test case (e.g., a simple catchment or benchmark configuration).
2. Configuring the model setup and FIAT test specification.
3. Running the test case through FIAT.

The emphasis here is on the **testing workflow**, not on any one model’s physics or numerical scheme. By separating the test definition from the model implementation, FIAT makes it easier to experiment with calibration test suites.

### Prerequisites

To follow along with this notebook, you should have FIAT installed in your Python environment (for example, via `pip` or from source) and be familiar with basic Python and Jupyter usage. The test case we use here is intentionally simple, so that the focus remains on understanding how FIAT orchestrates the run, rather than on complex data preparation.

In the next section, we will import FIAT, inspect the available test cases, and configure the one we want to run.

### Installation

FIAT is currently developed on GitHub and can be installed directly from the repository. To follow this notebook, first make sure you have a recent version of Python (e.g., 3.10+) and `pip` available, then run:

```console
$ pip install git+https://github.com/kasra-keshavarz/fiat.git@dev
```

This command installs FIAT from the `dev` branch of the repository, which is where the latest example notebooks and test workflows are being developed. After installation completes, you should be able to import FIAT in Python with:

```python
import fiatmodel
```

### Test case: Wolf Creek Research Basin with MESH and Ostrich

In this notebook, we focus on a specific application of FIAT: testing the **Wolf Creek Research Basin** using the **MESH** hydrological model and the **Ostrich** optimization framework. The aim is to demonstrate how FIAT can orchestrate a full calibration workflow, where a model (MESH) is repeatedly run under the control of an external optimizer (Ostrich) to improve model parameters against observational data.

FIAT creates a **calibration test case** that encapsulates all of the information needed for this workflow: the model configuration, the parameter space to explore, the objective function, and the links to the forcing and observation data for the Wolf Creek basin. Once this test case is defined, FIAT uses Ostrich’s internal optimization algorithms to systematically adjust the MESH parameters. For each candidate parameter set, FIAT triggers a MESH simulation, collects the performance metrics, and passes them back to Ostrich.

By iterating in this way, FIAT effectively wraps MESH inside an optimization loop driven by Ostrich. This allows us to treat the calibration as a reproducible, well-defined test case: one that can be rerun, compared across model versions or configurations, and extended to other basins with minimal changes to the workflow.

### Required tools: FIAT, MESHFlow, MESH, and Ostrich

In addition to FIAT itself, this workflow relies on the **MESHFlow** tooling as well as the **MESH** hydrological model and the **Ostrich** optimization framework. MESHFlow provides a set of utilities and workflows to set up and manage MESH model configurations, which FIAT then uses to define and run the Wolf Creek test case. Before running the cells below, ensure that MESHFlow is installed from its GitHub repository and that both MESH and Ostrich are available in your environment and properly configured on your system `PATH`.

You can install **MESHFlow** with:

```console
$ pip install git+https://github.com/CH-Earth/meshflow.git@main
```

Please also make sure the **MESH** executable (`sa_mesh`) and the **Ostrich** executable (`ostrich`) are installed and accessible in your system's PATH. You can download MESH from the [Environment and Climate Change Canada's GitHub repository](https://github.com/MESH-Model/MESH-Dev) and also separately, Ostrich from its own [GitHub repository](https://github.com/DOI-BOR/ostrich).

It will be up to the user to properly compile and install MESH and Ostrich on their system, as this process can vary depending on the operating system and environment. Once installed, ensure that you can run `sa_mesh` and `ostrich` commands from your terminal or command prompt to verify that they are correctly set up.

The **Ostrich** executable need to be copied where the calibration instance will be created, and **MESH** executables need to be available under the `model` directory of the calibration workflow.

### Importing required libraries

Now that FIAT and MESHFlow are installed, we can import the core Python libraries and the main workflow tools we will use in this notebook. These imports cover basic path handling, data analysis, plotting, and the FIAT/MESHFlow interfaces for setting up and running the Wolf Creek calibration test case.

In [1]:
# FIAT calibration workflow
import fiatmodel

# `xarray` to read the observation NetCDF file
import xarray as xr

### Setting up the FIAT `Calibration` object

With the basic environment ready, the next step is to define a **calibration test case** in FIAT by constructing a `Calibration` object (or the equivalent calibration configuration in FIAT’s API). Conceptually, this object encapsulates:

- The **model interface** (MESH, accessed via MESHFlow),
- The **optimizer** (Ostrich and its algorithm / configuration),
- The **parameter space** to be explored,
- The **objective function** and evaluation period,
- Any **runtime options** (e.g., maximum iterations, convergence criteria, random seeds).

At a minimum, you will need to specify:

- Which parameters are free to vary during calibration (and their bounds),
- How performance is measured (e.g., Nash–Sutcliffe efficiency, log-NSE, bias),
- Practical limits on the optimization (number of evaluations, etc.).

In the following cells, we will focus on describing the **parameter space** as required by FIAT for a MESH–Ostrich calibration. This parameter definition is passed into the `Calibration` object so that FIAT can hand off candidate parameter sets to MESHFlow, MESH, and ultimately Ostrich.

---

### Parameter dictionaries for MESH

To calibrate MESH, FIAT's MESH recipe expects parameter information to be organized into **three distinct parameter dictionaries**. Each dictionary is keyed by **computational unit identifiers**, and each key maps to a dictionary of parameters and their search bounds. The three dictionaries correspond to:

1. **CLASS parameters**  
2. **Hydrology parameters**  
3. **Routing parameters**

For the **CLASS** and **hydrology** parameter dictionaries, the computational unit is the **Group Response Unit (GRU)**. For the **routing** parameter dictionary, the computational units are the **river classes**, with a maximum of 5 distinct river classes.

#### Dictionary keys: computational unit indices

Each parameter dictionary uses **integer keys** starting from 1:

- For CLASS and hydrology parameters:
  - Key `1` corresponds to the first GRU,
  - Key `2` to the second GRU,
  - …
  - Key `N` to the last GRU, where `N` is the total number of GRUs defined for the model instance.

- For routing parameters:
  - Key `1` corresponds to the first river class,
  - Key `2` to the second river class,
  - …
  - Up to a maximum of **5** river classes (i.e., keys `1` through `5`, or fewer if the model uses fewer river classes).

This means that for each MESH instance, FIAT assumes a well-defined mapping between:

- GRU indices (`1..N`) and their physical meaning (land cover, soil type, etc.), and  
- River class indices (`1..R`) and their routing properties (`R ≤ 5`).

#### Dictionary values: parameter definitions and bounds

For **each computational unit**, the corresponding value in the dictionary is **another dictionary** that defines:

- The **parameter names** for that unit (e.g., `SDEP`, `QA50`, or hydrological/routing parameters specific to MESH), and  
- The **search bounds** for each parameter during calibration.

In practice, this often looks like a nested structure where:

- The outer dictionary is keyed by the computational unit index (GRU or river class number), and
- The inner dictionary maps each parameter name to its lower and upper bounds (and optionally additional metadata, depending on the FIAT API).

FIAT then uses these nested dictionaries to construct the calibration parameter space passed to Ostrich. For each Ostrich iteration, FIAT will:

1. Draw or receive a candidate parameter set within the specified bounds for each computational unit,
2. Write the corresponding MESH configuration files via MESHFlow,
3. Run MESH,
4. Evaluate the objective function, and
5. Return the performance metric to Ostrich.

In the following of the notebook, we will define these three parameter dictionaries with parameters we would like to calibrate for the Wolf Creek Research Basin setup and pass them into the FIAT `Calibration` object.

In [2]:
# defining MESH calibration parameter bounds
class_dict_bounds = {
    1: {
        'sdep': [0.5, 4.0],
    },
    5: {
        'sdep': [0.5, 4.0],
    },
}

hydrology_dict_bounds = {
    1: {
        'zsnl': [0.03, 0.6],
    },
    5: {
        'zsnl': [0.03, 0.6],
    },
}

routing_dict_bounds = {
    1: {
        'r2n': [0.001, 2.0],
        'r1n': [0.001, 2.0]
    },
    2: {
        'r2n': [0.001, 2.0],
        'r1n': [0.001, 2.0]
    },
    3: {
        'r2n': [0.001, 2.0],
        'r1n': [0.001, 2.0]
    },
    4: {
        'r2n': [0.001, 2.0],
        'r1n': [0.001, 2.0]
    },
    5: {
        'r2n': [0.001, 2.0],
        'r1n': [0.001, 2.0]
    },
}

To make the Wolf Creek calibration example concrete and practical, we now specify the actual MESH parameters and bounds that FIAT will expose to Ostrich. As described above, we construct three dictionaries—`class_dict_bounds`, `hydrology_dict_bounds`, and `routing_dict_bounds`—each keyed by the relevant computational unit index and each value being a dictionary of parameter names and their search bounds.

In this example, we intentionally keep the parameter set **small and interpretable**, focusing on a subset of randomly selected parameters for two GRUs (GRU 1 and GRU 5) and five river classes. This yields a compact but meaningful calibration problem that is well suited for demonstration.

### Constructing the `Calibration` object

With the parameter dictionaries in place, we can now instantiate the FIAT `Calibration` object. This object ties together four key components:

1. **Calibration software** (`calibration_software='ostrich'`)  
2. **Model software** (`model_software='mesh'`)  
3. A **calibration configuration** (`calibration_config`) that tells Ostrich how to search the parameter space  
4. A **model configuration** (`model_config`) that tells FIAT how to run MESH  
5. A list of **observations** that define what we are calibrating against


In [3]:
# Reading observed values
obs_obj = xr.open_dataset('./wolf-creek-research-basin/wolf-creek-gauge-data.nc')

# Creating a `Calibration` object
c = fiatmodel.Calibration(
    calibration_software = 'ostrich',
    model_software = 'mesh',
    calibration_config = {
        'instance_path': './wolf-creek-research-basin-calibration/',
        'random_seed': 10, # could be any integer!
        'algorithm': 'ParallelDDS', # Ostrich-specific algorithm keyword
        'algorithm_specs': { # refer to ostrich software manual for these keys
            'PerturbationValue': 0.2,
            'MaxIteration': 10_000,
            'UseRandomParamValue': None,
        },
        'spinup_start': '1980-01-01 12:00:00', # spin-up start date
        'dates': [ # one or more calibration dates, here only one
            {
                'start': '1980-01-01 23:00:00',
                'end': '1980-01-02 23:00:00',
            },
        ],
        'objective_functions': {
            'QO': {
                'kge_2012': ['-1 * alaska_72'], # KGE-2012 value calculated using `alaska_72` gauge data
            },
        },
    },
    model_config = {
        'instance_path': './wolf-creek-research-basin/',
        'parameter_bounds': {
            'class': class_dict_bounds,
            'hydrology': hydrology_dict_bounds,
            'routing': routing_dict_bounds,
        },
        'executable': 'sa_mesh', # to be added to the copying stuff + required_files
    },
    observations = [
        {
            "name": "alaska_72",
            "type": "QO", # the output of MESH to be compared with
            "timeseries": obs_obj['discharge'].isel(gauge_name=2).to_series(), # have a look at `obs_obj` separately yourself!
            "unit": "m^3/s", # unit of the observed values
            "computational_unit": "subbasin", # dimension of computational unit in MESH
            "computational_unit_id": 38, # this is up to user to specify the accurate position of observed values
            "freq": "1h", # frequency of the observed values, necessary due to potential missing values with observation records
        },
    ],
)

#### Calibration configuration (`calibration_config`)

In `calibration_config` we specify how FIAT should set up and control Ostrich:

- `instance_path`: location where FIAT will generate the complete calibration instance (Ostrich input files, MESH run scripts, etc.).
- `random_seed`: initialized with the current time to control random aspects of the optimization (e.g., starting points).
- `algorithm`: we choose the **ParallelDDS** algorithm, a parallelizable variant of DDS suited for exploring moderately sized parameter spaces.
- `algorithm_specs`: options passed directly to Ostrich, such as:
  - `PerturbationValue = 0.2`: controls how strongly parameters are perturbed during the search.
  - `MaxIteration = 10_000`: upper limit on the number of model evaluations / iterations.
  - `UseRandomParamValue = None`: here left as `None` (defer to Ostrich defaults or disable this option).

We also define the **temporal setup** of the calibration:

- `spinup_start = '1990-01-01 00:00:00'`  
  A spin-up period used to bring the model states to a realistic condition before the calibration period starts.
- `dates`: a list of one or more calibration windows. In this case we use:
  - Start: `1995-01-01 00:00:00`
  - End: `2005-12-31 23:00:00`  
  All objective function evaluations will be computed over this period.

The `objective_functions` block specifies what we are trying to optimize. Here:

- We define a discharge-type objective (`"QO"`),
- Use the **Kling–Gupta Efficiency (KGE 2012)** as the metric,
- And apply it to the observation named `alaska_72`, with a negative sign:
  - `'kge_2012': ['-1 * alaska_72']`  
  Multiplying by `-1` turns a maximization problem (larger KGE is better) into a minimization problem, which matches Ostrich’s default behavior.

#### Model configuration (`model_config`)

The `model_config` tells FIAT how to configure and run MESH:

- `instance_path = './wolf-creek-research-basin/mesh/'`  
  Points to the base MESH setup for the Wolf Creek Research Basin (forcing, parameter/template files, etc.).
- `parameter_bounds`: the three parameter dictionaries we defined earlier:
  - `'class': class_dict_bounds`
  - `'hydrology': hydrology_dict_bounds`
  - `'routing': routing_dict_bounds`  
  These define which MESH parameters are calibrated and their allowable ranges for each GRU or river class.
- `executable = 'sa_mesh'`  
  The MESH executable to be called by FIAT during each Ostrich iteration. FIAT will use this name when generating run scripts or system calls.

#### Observations

Finally, the `observations` list describes the time series against which we calibrate:

- `"name": "alaska_72"`  
  An internal identifier for this observation, used in the `objective_functions` definition.
- `"type": "QO"`  
  Indicates that this is a discharge (flow) observation, matching the `"QO"` objective function group.
- `"timeseries"`: a pandas `Series` created from an xarray dataset:
  - `xr.open_dataset(obs_path)['discharge'].isel(gauge_name=2).to_series()`  
  This pulls the discharge series for a specific gauge (index 2) from the observation dataset at `obs_path`.
- `"unit": "m^3/s"`  
  Specifies the physical unit of the observations.
- `"computational_unit": "subbasin"` and `"computational_unit_id": 38`  
  Map the observation to subbasin 38 in the MESH model, so FIAT knows which simulated discharge series to compare against `alaska_72`.
- `"freq": "1h"`  
  States that the time series is hourly, matching the temporal resolution expected by the model outputs and the objective function.

Together, this `Calibration` object fully specifies how FIAT will:

1. Generate a calibration instance on disk,
2. Use Ostrich (ParallelDDS) to explore the parameter space defined by our three dictionaries,
3. Run MESH via the `sa_mesh` executable,
4. Compare simulated discharge at subbasin 38 against the `alaska_72` gauge, and
5. Iterate until the KGE-based objective function is optimized over the 1995–2005 calibration period.


For full details on the entries of the `Calibration` object, please refer to the [FIAT documentation](https://fiatmodel.readthedocs.io/en/latest/).

### Last step

The last step is to prepare a calibration instance on disk:

In [4]:
c.prepare(output_path='./wolf-creek-research-basin-calibration/')

/Users/kasrakeshavarz/Documents/github-repos/fiatmodel/src/fiatmodel/calibration/optimizer.py:309: UserWarning: The directory ./wolf-creek-research-basin-calibration/etc/templates already exists. Contents may be overwritten.
  warnings.warn(f"The directory {path} already exists."
/Users/kasrakeshavarz/Documents/github-repos/fiatmodel/src/fiatmodel/calibration/optimizer.py:309: UserWarning: The directory ./wolf-creek-research-basin-calibration/etc already exists. Contents may be overwritten.
  warnings.warn(f"The directory {path} already exists."
